
# POC 激活验证

In [1]:
import dgl
import torch
from utils_gnn import cnn2graph_activation
# from model_lib import mnist_cnn_model as father_model
from utils_basic import load_spec_model
from utils_gnn import padding, unpadding
from dgl.data import DGLDataset
import os
from tqdm import tqdm
from dgl import save_graphs, load_graphs
import torch
import json
from torch import nn
import numpy as np
# from model_lib.mnist_cnn_model import Model
from random import randint
from utils_basic import load_spec_model
from sklearn.metrics import roc_auc_score
from torchinfo import summary
import torch.nn.functional as F
from dgl.nn.pytorch.conv import GINConv
from dgl.nn.pytorch.glob import SumPooling, AvgPooling, MaxPooling, SortPooling
from utils_gnn import MLP


# x = '/home/dorian/repos/Meta-Nerual-Trojan-Detection/shadow_model_ckpt/mnist/models5/shadow_jumbo_9.model'
x = './shadow_model_ckpt/mnist/models5/shadow_jumbo_0.model'
# load model 
# Model = load_spec_model(father_model, '5')
from model_lib.mnist_cnn_model import Model6 as Model
model = Model(gpu=True)
params = torch.load(x)
model.load_state_dict(params)
del params

# load model detail 
model_detail = {}
model_detail_path = "./intermediate_data/model_detail.json"
import json
with open(model_detail_path, 'r') as f:
    model_detail = json.load(f)
# print(model_detail)
g = cnn2graph_activation(model, model_detail['mnist']['5'])
dgl.save_graphs('./intermediate_data/grapj_test.bin', g)
del model_detail

import torch
import torchvision
import torchvision.transforms as transforms
# from utils_gnn import SGNACT
GPU = True
if GPU:
        torch.cuda.manual_seed_all(0)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
transform = transforms.Compose([
            transforms.ToTensor(),
        ])
BATCH_SIZE = 1
# MNIST image dataset 
trainset = torchvision.datasets.MNIST(root='./raw_data/', train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE)

# get a image
image = None
label = None
for i, (x_in, y_in) in enumerate(dataloader):
    image = x_in
    x, params = model(image)
    with open("./intermediate_data/params.json", 'w') as f:
        json.dump(params, f)
    label = y_in
    break
del trainset, dataloader

In [2]:
# define the init process
def init_conv(data, data_size, weight, bias, kernel_size, stride, padding):
    row, col = weight.size()
    # print(weight.size())
    # get actual conv kernel weight and bias
    w = unpadding(weight, kernel_size[0], kernel_size[1])
    w = w.unsqueeze(0).unsqueeze(0)
    b = unpadding(bias, 1, 1)[0]
    # get conv operator 
    # print("kernel_size, stride, padding:")
    # print(kernel_size, stride, padding)
    operator = torch.nn.Conv2d(1, 1, kernel_size=kernel_size, 
                    stride=stride, padding=padding)
    # set conv operator weight and bias
    operator.weight.data = w
    operator.bias.data = b
    # conduct conv operation
    # print("conv input size:", data.size())
    x = operator(data.to("cuda"))

    return x


# def maxpool(kernel_size, stride, padding):
#     pass  

# def message_func(edges):
#     # print("this is message function.")
#     # print("message fucntion ends")
#     # print("edges src ft size:", edges.src['ft'].size())
#     return {'m': edges.src['ft']}
#     # return {}


# def reduce(nodes):
#     # print("this is reduce function")

#     pre_data = nodes.mailbox['m']
#     # print("mailbox data size:", pre_data.size())
#     ft = torch.zeros((1, 1, 28, 28)).to("cuda")
#     # print("nodes data size in reduce:", ft.size())
#     # print("nodes ft size:", nodes.data['ft'].size())
#     if pre_data.all() == 0:
#         return {'h': torch.sum(pre_data, dim=1)}
#     for i in range(len(pre_data)):
#         # else do conv operation
#         kernel_size, stride, padding = nodes.data['params'][i]

#         res_ft = my_conv(pre_data[i][0], None, nodes.data['x'][i], nodes.data['bias'][i], kernel_size, stride, padding)
#         # print("res_ft in reduce func:", res_ft)
#         ft += res_ft
#     # print("size before and after:", nodes.mailbox['m'].size(), ft.size())
#     # print("reduce function ends")
#     return {'h': ft}


def initiate_node_feature(graph, image):
    ft = None 
    mask = graph.ndata['tag'] == 0
    out_channels = int(sum(mask))
    ft = torch.zeros((len(graph.nodes()), 1, 1, 28, 28))
    convd_size = torch.zeros((len(graph.nodes()), 2))
    for i in range(out_channels):
        kernel_size, stride, padding = graph.ndata['kernel_params'][i]
        # do conv
        res_ft = init_conv(image, None, graph.ndata['kernel_weight'][i], graph.ndata['bias'][i],
                kernel_size, stride, padding)
        # do relu
        relu_opt = torch.nn.functional.relu
        res_ft = relu_opt(res_ft)

        # do max pooling
        pooling = graph.ndata['pooling_params'][i]
        if pooling.all() != 0:
            # do max_pooling
            kernel_size, stride, pad, dilation, ceil_mode = pooling
            max_pooling_operator = torch.nn.MaxPool2d(kernel_size=kernel_size, stride=stride, 
                padding=pad, dilation=dilation, ceil_mode=ceil_mode)
            
            res_ft = max_pooling_operator(res_ft)
        _, _, r, c = res_ft.size()
        ft[i] = res_ft
        convd_size[i] = torch.tensor([int(r), int(c)])
    graph.ndata['ft'] = ft.to("cuda")
    graph.ndata['ft_size'] = convd_size.to("cuda")


# def conv_message_passing(graph):
#     # get node id first, then get subgraph
#     def nodes_feature_smaller_than_one(nodes):
#         return nodes.data['tag'] <= 1
    
#     nodes_idx = graph.filter_nodes(nodes_feature_smaller_than_one)
#     # print("in conv mp, nodes idx:", nodes_idx)
#     subg = dgl.node_subgraph(graph, nodes_idx)
#     def conv_reduce_func():
#         pass
#     subg.update_all(dgl.function.copy_src('ft', 'm'))
#     ft = subg.ndata['ft']

# def cnn_cal(graph, image):
#     initiate_node_feature(graph, image)
#     # conv_message_passing(graph)
#     # graph.update_all(message_func=message_func, reduce_func=reduce_func)
#     #ft = graph.ndata['ft'][0]
#     #return ft

# import json
# # get cnn results
# pred, params = model(x_in)
# print(params.keys())
# with open("./intermediate_data/params.json", "w") as f:
#     json.dump(params, f)
# # get gnn transmission results
# with torch.no_grad():
#     res = cnn_cal(g, image)


In [3]:
g.ndata['tag']

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3],
       device='cuda:0')

In [4]:
# init ft feature and get subg
with torch.no_grad():
    initiate_node_feature(g, image)
print(g.ndata['tag'])
# get node id
def nodes_with_feature_smaller_two(nodes):
    return nodes.data['tag'] <= 1
nodes_idx = g.filter_nodes(nodes_with_feature_smaller_two)
# print(nodes_idx)
subg = dgl.node_subgraph(g, nodes_idx, relabel_nodes=True)
print(subg.nodes())
ft32 = None
# got the right subg of conv layers

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3],
       device='cuda:0')
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87],
       device='cuda:0')


In [5]:
subg.ndata['ft']

tensor([[[[[0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           ...,
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070]]]],



        [[[[0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           ...,
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206]]]],



        [[[[0.1635, 0.1635, 0.1635,  ..., 0.1635, 0.1635, 0.1635],
           [0.1635, 0.1635, 0.1635,  ..., 0.1635, 0.1635, 0.1635],
           [0.1635

In [6]:
subg.ndata['ft']

tensor([[[[[0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           ...,
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070]]]],



        [[[[0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           ...,
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206]]]],



        [[[[0.1635, 0.1635, 0.1635,  ..., 0.1635, 0.1635, 0.1635],
           [0.1635, 0.1635, 0.1635,  ..., 0.1635, 0.1635, 0.1635],
           [0.1635

In [7]:
subg.ndata['ft_size']

tensor([[28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [28., 28.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],


In [8]:
print(subg.ndata['ft'])
print(subg.ndata['ft_size'])

tensor([[[[[0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           ...,
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070],
           [0.1070, 0.1070, 0.1070,  ..., 0.1070, 0.1070, 0.1070]]]],



        [[[[0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           ...,
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206],
           [0.0206, 0.0206, 0.0206,  ..., 0.0206, 0.0206, 0.0206]]]],



        [[[[0.1635, 0.1635, 0.1635,  ..., 0.1635, 0.1635, 0.1635],
           [0.1635, 0.1635, 0.1635,  ..., 0.1635, 0.1635, 0.1635],
           [0.1635

In [9]:
def my_reduce(nodes):
    print("my reduce function called!")
    m = nodes.mailbox['m']
    if torch.eq(m, torch.zeros(m.size()).to("cuda")).all() == True:
        print("received message all are zeros! qaq")
    actual_data_size = nodes.mailbox['ret_size']

    print("mailbox data size:", m.size(), actual_data_size.size())
    print(nodes)
    mshape = m.size()
    n, f, d = actual_data_size.size()
    print("n, d, f:", n, f, d)
    ret_size = torch.zeros((n, d)).to("cuda")
    ret_ft = torch.zeros((n, 1, 1, 28, 28)).to("cuda")
    pre_ft = None
    pre_weight = None
    cnt = 0 
    print("received feature size", m.size())
    print("received data size:", actual_data_size.size(), actual_data_size[0])
    # reduce nodes batch N
    for out_idx in range(mshape[0]):
        channels = mshape[1]
        # begin the process for each node
        received_data = m[out_idx]
        if torch.eq(received_data, torch.zeros(received_data.size()).to("cuda")).all() == True:
            print("skiped %d feats." % cnt)
            cnt += 1
            continue
        # prepare data -> [1, channels, row, col]
        received_data_size = actual_data_size[out_idx]
        print("received_data_size: ", received_data_size)
        row, col = received_data_size[0]
        row, col = int(row), int(col)
        # (n, 1, 1, 28, 28) -> (1, n, x, x)
        prepared_data = torch.zeros((1, channels, row, col))
        for inner_idx in range(len(received_data)):
            data = received_data[inner_idx].reshape(28, 28)
            if data.all() == 0.:
                continue
            print("inner data size:", data.size())
            data = unpadding(data, row, col)
            prepared_data[0][inner_idx] = data
        print("prepared data size: ", prepared_data.size())    

        # prepare conv kernel weight [1, channels, w, h] and bias
        _, width, height = nodes.data['kernel_size'][out_idx]
        kernel_weight = nodes.data['kernel_weight'][out_idx]
        print("kernel weight data size:", kernel_weight.size())
        kernel_weight = unpadding(kernel_weight, channels, width * height)
        kernel_weight = kernel_weight.reshape(1, channels, width, height)
        print("reshaped kernel weight size:", kernel_weight.size())
        bias = nodes.data['bias'][out_idx]
        kernel_bias = unpadding(bias, 1, 1)[0]
        
        # prepare conv operator
        kernel_size, stride, pad = nodes.data['kernel_params'][out_idx]
        # print("idx:", out_idx, nodes.data['params'][out_idx])
        conv_operator = torch.nn.Conv2d(channels, 1, kernel_size=kernel_size, stride=stride, padding=pad)
        print("conv kernel size: ", conv_operator.weight.data.shape)
        # get and set my_conv weight & bias
        weight = nodes.data['kernel_weight'][out_idx]
        if pre_weight is None:
            pre_weight = weight
        elif torch.eq(pre_weight, weight).all() == True:
            print("this weight is same as pre weight totally!")
        else:
            print("weight diffs!")

        # print("weight size:", conv_operator.weight.size())
        conv_operator.weight.data = kernel_weight
        # print("weight size:", conv_operator.weight.size())
        conv_operator.bias.data = kernel_bias

        # do conv
        ft = conv_operator(prepared_data.to("cuda")).to("cuda")

        # do relu
        relu_operator = torch.nn.functional.relu
        ft = relu_operator(ft)

        # do max_pooling
        pooling_params = nodes.data['pooling_params'][out_idx]
        if pooling_params.any() != 0:
                # do max_pooling
                kernel_size, stride, pad, dilation, ceil_mode = pooling_params
                kernel_size, stride, pad, dilation, ceil_mode = int(kernel_size), int(stride), int(pad), int(dilation), bool(ceil_mode)
                max_pooling_operator = torch.nn.MaxPool2d(kernel_size=kernel_size, stride=stride, 
                    padding=pad, dilation=dilation, ceil_mode=ceil_mode)
                
                ft = max_pooling_operator(ft)
            
        print("ft size:", ft.size())
        print("ft is all zero: ", ft.all() == 0.)
        print(ft)
        # a_break = input()
        if ft is None:
            print("ft is None")
        _, _, row, col = ft.size()
        ft = ft.reshape(row, col)
        print("ft reshaped size:", ft.size())
        ft = padding(ft, 28, 28)
        ft = ft.reshape(1, 1, 28, 28)
        ft_size = torch.tensor([row, col])
        ret_size[out_idx] = ft_size.to("cuda")
        ret_ft[out_idx] = ft.to("cuda")
        if torch.eq(ft, torch.zeros(ft.size()).to("cuda")).all() == True:
            print("idx: ", out_idx)
            # a_break = input()
        # else:
            # print(ft)
            # a_break = input()
        if pre_ft is None:
            pre_ft = ft
        elif torch.eq(pre_ft, ft).all() == True:
            print("this ft is equals pre ft totally!")
        else:
            print("final feat diffs!")
        pre_ft = ft
        # a_break = input()

    # return size is [n, 1, 1, 28, 28], reduced from [n, m, 1, 1, 28, 28]
    return {'h': ret_ft, 'reduce_ret_size': ret_size}

def my_message(edges):
    return {'m': edges.src['ft'], 'ret_size': edges.src['ft_size']}

def print_zeros_in_feat(key):
    print(key + ": ")
    zeros = []
    nonzero = []
    for id in subg.nodes():
        id = int(id)
        feat = subg.nodes[id].data[key]
        if torch.eq(feat, torch.zeros(feat.size()).to("cuda")).all() == True:
            zeros.append(id)
        else:
            nonzero.append(id)
    print("zeros - ", zeros)
    print("nonzeros - ", nonzero)


with torch.no_grad():
    subg.update_all(my_message, my_reduce)
    print("ret_size of data: ", subg.ndata['reduce_ret_size'])
    print("convd_size of data: ", subg.ndata['ft_size'])
    print("subg ft_size:", subg.ndata['ft_size'])
    print("subg ret_size:", subg.ndata['reduce_ret_size'])
    subg.ndata['ft_size'] += subg.ndata['reduce_ret_size']
    print_zeros_in_feat('ft')
    print_zeros_in_feat('h')
    subg.ndata['ft'] += subg.ndata['h']
    print_zeros_in_feat('ft')
    print_zeros_in_feat('h')
    cur_ft32 = subg.ndata['ft'][32]
    if ft32 is not None and torch.eq(cur_ft32, ft32).all():
        ft32 = cur_ft32
        print("ft32 equals!")
    else:
        ft32 = cur_ft32
        print("ft32 not equeal!")



my reduce function called!
received message all are zeros! qaq
mailbox data size: torch.Size([8, 16, 1, 1, 28, 28]) torch.Size([8, 16, 2])
n, d, f: 8 16 2
received feature size torch.Size([8, 16, 1, 1, 28, 28])
received data size: torch.Size([8, 16, 2]) tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], device='cuda:0')
skiped 0 feats.
skiped 1 feats.
skiped 2 feats.
skiped 3 feats.
skiped 4 feats.
skiped 5 feats.
skiped 6 feats.
skiped 7 feats.
my reduce function called!
mailbox data size: torch.Size([48, 32, 1, 1, 28, 28]) torch.Size([48, 32, 2])
n, d, f: 48 32 2
received feature size torch.Size([48, 32, 1, 1, 28, 28])
received data size: torch.Size([48, 32, 2]) tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],

In [20]:
torch.eq(subg.ndata['ft'][0], conv1[0][0].to("cuda"))

tensor([[[[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
           False, False, False, False, False, False, False, False, False, False,
           False, False, False, False, False, False,  True,  True],
          [ True,  True,  True,  True,  True,  True, False, False, False, False,

In [10]:
print(subg.ndata['ft'].size())

torch.Size([88, 1, 1, 28, 28])


In [11]:
cnn_params = None 
with open("./intermediate_data/params.json", "r") as f:
    cnn_params = json.load(f)

In [12]:
ft = g.ndata['ft']
mask = g.ndata['tag'] == 0
print(ft[mask].size())
model_out = ft[mask]
out = torch.tensor(cnn_params['conv1'])
print(out.size())
model_out = model_out.resize_((32, 28, 28)).to("cuda")
out = out.resize_((32, 28, 28)).to("cuda")
print(model_out.size(), out.size())
# conv1 out is the same 
from torch.nn.functional import relu
print("conv1 out is the same: ", torch.eq(model_out, relu(out)).all() == True)

torch.Size([32, 1, 1, 28, 28])
torch.Size([1, 32, 28, 28])
torch.Size([32, 28, 28]) torch.Size([32, 28, 28])
conv1 out is the same:  tensor(False, device='cuda:0')


In [13]:
# checking output by gnn
from torch.nn.functional import relu
from utils_gnn import unpadding
conv1 = torch.tensor(cnn_params['conv1'])
conv2 = torch.tensor(cnn_params['conv2'])
conv3 = torch.tensor(cnn_params['conv3'])
conv4 = torch.tensor(cnn_params['conv4'])

ft = subg.ndata['ft']
print("ft shape:", ft.shape)
ft_size = subg.ndata['ft_size']
print(conv1.size(), conv2.size(), conv3.size(), conv4.size())
dict_size = dict()
for s in ft_size:
    if str(s) not in dict_size:
        dict_size[str(s)] = 1
    else:
        dict_size[str(s)] += 1
print(dict_size)


ft shape: torch.Size([88, 1, 1, 28, 28])
torch.Size([1, 32, 28, 28]) torch.Size([1, 32, 14, 14]) torch.Size([1, 16, 14, 14]) torch.Size([1, 8, 7, 7])
{"tensor([28., 28.], device='cuda:0')": 32, "tensor([14., 14.], device='cuda:0')": 32, "tensor([0., 0.], device='cuda:0')": 24}


In [15]:
# for idx in range(len(gnn2_out)):
#     w, h = subg.ndata['ft_size'][idx]
#     w, h = int(w), int(h)
#     dst_out[idx] = unpadding(gnn2_out[idx], w, h)
# print("dst_out size:", dst_out.size())

# print(conv2_out.size(), gnn2_out.size())
# import numpy as np
# for id in range(len(conv2_out)):
#     np.savetxt("./intermediate_data/cnn_out_%d.csv" % id, conv2_out[id].cpu().numpy(), delimiter=',')
#     np.savetxt("./intermediate_data/gnn_out_%d.csv" % id, gnn2_out[id].cpu().numpy(), delimiter=',')

In [25]:
# weight of model
cnt = 0
conv1_weight = model.conv1.weight.data
for weight in conv1_weight:
    _, w, h = weight.size()
    w, h = int(w), int(h)
    np.savetxt("./intermediate_data/model_weight/conv1_%d.csv" % cnt, weight.reshape(w, h).cpu().numpy(), delimiter=',')
    cnt += 1
conv2_weight = model.conv2.weight.data
for weight in conv1_weight:
    _, w, h = weight.size()
    w, h = int(w), int(h)
    np.savetxt("./intermediate_data/model_weight/conv2_%d.csv" % cnt, weight.reshape(w, h).cpu().numpy(), delimiter=',')
    cnt += 1
conv3_weight = model.conv3.weight.data
for weight in conv1_weight:
    _, w, h = weight.size()
    w, h = int(w), int(h)
    np.savetxt("./intermediate_data/model_weight/conv3_%d.csv" % cnt, weight.reshape(w, h).cpu().numpy(), delimiter=',')
    cnt += 1
conv4_weight = model.conv4.weight.data
for weight in conv1_weight:
    _, w, h = weight.size()
    w, h = int(w), int(h)
    np.savetxt("./intermediate_data/model_weight/conv4_%d.csv" % cnt, weight.reshape(w, h).cpu().numpy(), delimiter=',')
    cnt += 1

In [30]:
# weight of graph
kernel_weight = g.ndata['kernel_weight']
kernel_size = g.ndata['kernel_size']
for weight, size in zip(kernel_weight, kernel_size):
    c, w, h = size
    c, w, h = int(c), int(w), int(h)
    print(weight.size(), c, w, h)
    break

torch.Size([512, 512]) 1 5 5
